In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install plotly-geo
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)


In [ ]:
df = pd.read_csv("../input/new-york-times-covid19-dataset/us-states.csv")
print(df.head(5))
fip = pd.read_csv("../input/statefips/state_fips.csv")
fip = fip.loc[:, ["state_name","state_abbr","fips"]]
print(fip.head(5))

In [ ]:
df_state_avg = pd.read_csv("../input/new-york-times-covid19-dataset/us-states_1.csv")
print(df_state_avg.head(5))


In [ ]:
df_national_avg = pd.read_csv("../input/new-york-times-covid19-dataset/us.csv")
print(df_national_avg.tail(10))


In [ ]:
def fig_layout():
    fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     step="day",
                     stepmode="backward"),
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
    )
)

In [ ]:
fig = px.line(df_national_avg, x="date", y="cases", color='geoid', height=800, width=1200, title="Daily Case Count" )
fig_layout()
fig.show()
fig = px.line(df_national_avg, x="date", y="deaths", color='geoid', height=800, width=1200, title="Daily Death Count")
fig_layout()
fig.show()
fig = px.line(df_national_avg, x="date", y="cases_avg_per_100k", color='geoid', height=800, width=1200,title="Average Case Count per 100000")
fig_layout()
fig.show()
fig = px.line(df_national_avg, x="date", y="deaths_avg_per_100k", color='geoid', height=800, width=1200, title="Average Death Count per 100000")
fig_layout()
fig.show()

In [ ]:
fig = px.line(df_state_avg, x="date", y="cases", color='state', height=800, width=1200, title="State Case Count")
fig_layout()
fig.show()
fig = px.line(df_state_avg, x="date", y="deaths", color='state', height=800, width=1200, title="State Death Count")
fig_layout()
fig.show()

In [ ]:
import plotly.express as px
fig = px.line(df_state_avg, x="date", y="cases_avg_per_100k", color='state', height=800, width=1200, title="Average Case Count per 100000")
fig_layout()
fig.show()
fig = px.line(df_state_avg, x="date", y="deaths_avg_per_100k", color='state', height=800, width=1200, title="Average Death Count per 100000")
fig_layout()
fig.show()

In [ ]:
df2021=df.loc[(df['date'] > '2020-12-31')]

In [ ]:

fig = px.scatter(df2021, x="cases", y="deaths", animation_frame="date", animation_group="fips",
           size="deaths", color="cases", hover_name="state",
           log_x=False, size_max=100, range_x=[-500000, 5000000], range_y=[-200000,200000])

fig.show()

In [ ]:

dfcr = pd.read_csv("../input/new-york-times-covid19-dataset/us-counties-recent.csv")
print(dfcr.head(5))
dfcr.isnull().sum()
dfcr["fips"] = dfcr["fips"].fillna(value=dfcr["fips"].mean())
dfcr.isnull().sum()
dfcr = pd.merge(dfcr, fip, how='left', left_on='state', right_on='state_name')
print(dfcr.head(5))

In [ ]:
dfc = pd.read_csv("../input/new-york-times-covid19-dataset/us-counties.csv")
print(dfc.head(5))
dfc.isnull().sum()
dfc["fips"] = dfc["fips"].fillna(value=dfc["fips"].mean())
dfc.isnull().sum()

In [ ]:
df.isnull().sum()
df["fips_x"] = df["fips"].fillna(value=df["fips"].mean())
df.isnull().sum()

In [ ]:
px.choropleth(dfcr, 
              locations = 'state_abbr',
              color="cases", 
              animation_frame="date",
              color_continuous_scale="Inferno",
              locationmode='USA-states',
              scope="usa",height=800, width=1200,
              range_color=(0, 50000),
              title='Total COVID Cases Animation',
             )

In [ ]:
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings("ignore")

colorscale = [
    'rgb(239,239,239)',
    'rgb(195, 196, 222)',
    'rgb(163, 163, 163)',
    'rgb(144,148,194)',
    'rgb(101,104,168)',
    'rgb(80,239,239)',
    'rgb(60, 196, 222)',
    'rgb(40,148,194)',
    'rgb(20,104,168)',
    'rgb(5, 53, 132)'
]

uniquelist = dfcr.state.unique()
for state in uniquelist[5:10]:
    dfcr_st = dfcr[dfcr['state'] == state]
    dfcr_st['fips_x'] = dfcr_st['fips_x'].astype(int)
    dfcr_st['fips_x'] = dfcr_st['fips_x'].astype(str)
    fips = dfcr_st['fips_x'].tolist()
    values = dfcr_st['cases'].astype(int).tolist()
    endpts = list(np.mgrid[min(values):max(values):8j])
    title='USA COVID-19 Cases by County - ' + state
    legend_title = 'COVID-19 Cases Number ' + state
    fig = ff.create_choropleth(
        fips=fips, values=values,
        binning_endpoints=endpts,
        scope=[state],
        colorscale=colorscale,
        height=800, width=1200,
        county_outline={'color': 'rgb(15,15,55)', 'width': 1},
        state_outline={'color': 'rgb(15,45,55)', 'width': 1},
        show_state_data=True,
        show_hover=True, 
        centroid_marker={'opacity': 0},
        asp=2.9, 
        title=title,
        legend_title=legend_title,
    )
    fig.layout.template = None
    fig.show()
    print(state)
    print('Done')
    print('-'*30)